# **Capstone Project Computer Vision**

# Finger Detection and counting

# Import Libraries

In [19]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
# Được dùng cho việc tính toán khoảng cách.
from sklearn.metrics import pairwise

# **Global Variables**
Chúng ta sẽ sử dụng các biến global này sau này

In [20]:
#  Biến nền này sẽ là một biến toàn cầu mà chúng ta sẽ cập nhật thông qua một số hàm.
background = None

#  Bắt đầu với một giá trị trung gian nằm giữa 0 và 1 của trọng số tích lũy.
accumulated_weight = 0.5

# Thiết lập g vùng quan tâm (ROI) để nắm bắt tay.
roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

## **Finding Average Background Value**

In [21]:
def calc_accum_avg(frame, accumulated_weight):
  """
  Với một frame và một accumulated_weight trước đó, tính giá trị trung bình(weighted average) có trọng số của hình ảnh được truyền vào.
  """

  # Lấy nền background
  global background

  # Trước tiên tạo nền background cho bản sao của khung hình
  if background is None:
    background = frame.copy().astype("float")
    return None
  # Tính giá trị trung bình tích luỹ và cập nhật nền
  cv2.accumulateWeighted(frame, background, accumulated_weight)

# **Segment the Hand Region in Frame**

In [22]:
def segment(frame, threshold=25):

  global background

  # Tính toán Độ Khác Biệt Tuyệt Đối giữa nền và khung hình được truyền vào
  diff = cv2.absdiff(background.astype("uint8"), frame)

  # Áp dụng ngưỡng cho hình ảnh để chúng ta có thể nắm bắt phần trước cảnh
  # Chúng ta chỉ cần giá trị thresholded, vì vậy chúng ta sẽ loại bỏ mục đầu tiên trong bộ tổ hợp với dấu gạch dưới "_"
  _ , thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)

  # Lấy các đường viền ngoại cảnh từ hình ảnh
  # Một lần nữa, chúng ta chỉ lấy những gì chúng ta cần ở đây và loại bỏ phần còn lại
  contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Nếu độ dài danh sách đường viền là 0, thì chúng ta sẽ không lấy bất kỳ đường viền nào!
  if len(contours) == 0:
    return None
  else:
    # Đường viền ngoại cảnh lớn nhất là bàn tay (lớn nhất về diện tích)
    # Đây sẽ là phân đoạn của chúng ta
    hand_segment = max(contours, key = cv2.contourArea)

    # Trả về phân đoạn bàn tay và hình ảnh bàn tay đã ngưỡng hoá được
    return (thresholded, hand_segment)

In [23]:

def count_fingers(thresholded, hand_segment):

  # Tính toán Convex Hull của hand_segment(phân đoạn của bàn tay)
  conv_hull = cv2.convexHull(hand_segment)

  # Bây giờ Convex Hull sẽ có ít nhất 4 điểm ngoài cùng, ở trên, dưới, trái và phải.
  # Ví dụ conv_hull nó sẽ trả về dạng như này: [[[234 202]], [[ 51 202]], [[ 51 79]], [[234 79]]]

  # Tìm điểm trên, dưới, trái và phải.
  # Sau đó đảm bảo rằng chúng ở định dạng tuple
  top    = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
  bottom = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
  left   = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
  right  = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])

  # Theo lý thuyết, tâm của bàn tay nằm ở giữa đỉnh và đáy cũng như ở giữa trái và phải
  cX = (left[0] + right[0]) // 2  # Lấy toạ độ x của trái cộng phải rồi chia cho 2 lấy phần nguyên
  cY = (top[1] + bottom[1]) // 2  # Lấy toạ độ y của trên cộng dưới rồi chia cho 2 lấy phần nguyên

  # Tìm khoảng cách Euclidean lớn nhất giữa tâm của lòng bàn tay
  # và các điểm cực đại trên Convex Hull

  #Tính toán khoảng cách Euclidean giữa tâm của bàn tay và các điểm bên trái, bên phải, đỉnh và đáy.
  distance = pairwise.euclidean_distances([(cX,cY)], Y=[left,right,top,bottom])[0]

  #Lấy khoảng cách lớn nhất
  max_distance = distance.max()

  # Tạo một vòng tròn với bán kính là 90% của khoảng cách Euclidean lớn nhất này.
  radius = int(0.8 * max_distance)
  # Tính chu vi của vòng tròn này
  circumference = (2 * np.pi * radius)

  # Không lấy vùng quan tâm (ROI) chỉ trong hình tròn
  circular_roi = np.zeros(thresholded.shape[:2], dtype="uint8")

  # Vẽ hình tròn quanh vùng quan tâm
  cv2.circle(circular_roi, (cX, cY), radius, 255, 10)

  # Sử dụng phép bitwise AND với hình tròn quan tâm làm mặt nạ.
  # Điều này trả về phần được cắt bằng cách sử dụng mặt nạ trên hình ảnh ngưỡng của bàn tay.
  circular_roi = cv2.bitwise_and(thresholded, thresholded, mask=circular_roi)

  # Lấy các đường viền trong vùng tròn quan tâm
  contours, hierarchy = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

  # Đếm ngón tay bắt đầu từ 0
  count = 0

  # Duyệt qua các đường viền để xem có thể đếm thêm ngón tay nào không.
  for cnt in contours:
    # Hình chữ nhật bao quanh đường viền
    (x, y, w, h) = cv2.boundingRect(cnt)

    # Tăng số lượng ngón tay dựa trên hai điều kiện:

    # 1. Vùng đường viền không nằm ở phía dưới cùng của vùng bàn tay (cổ tay)
    out_of_wrist = ((cY + (cY * 0.25)) > (y + h))

    # 2. Số điểm dọc theo đường viền không vượt quá 25% chu vi của vùng tròn quan tâm (nếu không, chúng ta đang đếm điểm nằm ngoài bàn tay)
    limit_points = ((circumference * 0.25) > cnt.shape[0])

    if out_of_wrist and limit_points:
      count += 1
  return count

# **Run Project**

In [ ]:
# Mở kết nối với camera
cam = cv2.VideoCapture(1)

# Khởi tạo biến đếm số khung hình
num_frames = 0

# Lặp vô tận cho đến khi người dùng thoát khỏi chương trình
while True:
  # Lấy khung hình hiện tại
  ret, frame = cam.read()

  # Lật khung hình để không hiển thị hình ảnh đối xứng
  frame = cv2.flip(frame, 1)

  # Sao chép khung hình
  frame_copy = frame.copy()

  # Lấy vùng quan tâm (ROI) từ khung hình
  roi = frame[roi_top:roi_bottom, roi_right:roi_left]

  # Áp dụng chế độ xám và làm mờ cho ROI
  gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
  gray = cv2.GaussianBlur(gray, (7,7), 0)

  # Trong 30 khung hình đầu tiên, chúng ta sẽ tính trung bình của nền.
  # Chúng ta sẽ thông báo cho người dùng khi đang thực hiện việc này
  if num_frames < 60:
    calc_accum_avg(gray, accumulated_weight)
    if num_frames <= 59:
      cv2.putText(frame_copy, "Vui lòng chờ, đang lấy nền trung bình.", (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
      cv2.imshow("Finger Count",frame_copy)
  else:

    # Bây giờ chúng ta đã có nền, chúng ta có thể phân đoạn bàn tay.

    # Phân đoạn vùng bàn tay
    hand = segment(gray)

    # Kiểm tra trước tiên xem chúng ta có thể phát hiện một bàn tay không
    if hand is not None:

      # Giải nén
      thresholded, hand_segment = hand

      # Vẽ đường viền xung quanh phần bàn tay
      cv2.drawContours(frame_copy, [hand_segment + (roi_right, roi_top)], -1, (255,0,0), 1)

      # Đếm số ngón tay
      fingers = count_fingers(thresholded, hand_segment)

      # Hiển thị số lượng ngón tay
      cv2.putText(frame_copy, str(fingers), (70,45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

      # Hiển thị cả hình ảnh được ngưỡng
      cv2.imshow("Thresholded", thresholded)

  # Vẽ hình chữ nhật ROI trên bản sao khung hình
  cv2.rectangle(frame_copy, (roi_left, roi_top), (roi_right, roi_bottom), (0,0,255),5)

  # Tăng số khung hình để theo dõi
  num_frames += 1

  # Hiển thị khung hình với bàn tay được phân đoạn
  cv2.imshow("Finger Count", frame_copy)

  # Đóng cửa sổ bằng cách nhấn Esc
  k = cv2.waitKey(1) & 0xFF

  if k == 27:
    break

# Giải phóng camera và đóng tất cả cửa sổ
cam.release()
cv2.destroyAllWindows()